In [ ]:
!pip install SPARQLWrapper requests networkx matplotlib tqdm altair pandas pyspark
!wget http://www.philippe-fournier-viger.com/spmf/spmf.jar

--2024-04-12 17:44:39--  http://www.philippe-fournier-viger.com/spmf/spmf.jar
Resolving www.philippe-fournier-viger.com (www.philippe-fournier-viger.com)... 172.67.193.154, 104.21.33.228, 2606:4700:3035::ac43:c19a, ...
Connecting to www.philippe-fournier-viger.com (www.philippe-fournier-viger.com)|172.67.193.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12692580 (12M) [application/java-archive]
Saving to: ‘spmf.jar.2’

spmf.jar.2          100%[===================>]  12.10M  --.-KB/s    in 0.1s    

2024-04-12 17:44:39 (89.5 MB/s) - ‘spmf.jar.2’ saved [12692580/12692580]



On extrait les données de l'endpoint SPARQL de DBpedia et enregistre les transactions dans un fichier CSV.

In [ ]:
import os
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict
from itertools import combinations, chain
from operator import itemgetter
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import altair as alt
import time

DBPEDIA_ENDPOINT = 'https://dbpedia.org/sparql'
WIKIDATA_ENDPOINT = 'https://query.wikidata.org/sparql'
CLASS = 'dbo:Person'
SPMF_PATH = "spmf.jar"

def extract_data(limit=10000):
    transactions = []  # Liste pour stocker les transactions
    offset = 0  # Décalage initial pour la pagination
    while True:
        print(f"Extracting data from offset {offset}...")
        sparql = SPARQLWrapper(DBPEDIA_ENDPOINT)  # Initialisation de l'interface SPARQL
        query = f"""
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT ?entity (GROUP_CONCAT(DISTINCT ?predicate; separator=",") AS ?itemset)
        WHERE {{
            ?entity a {CLASS} .
            ?entity ?predicate ?object .
            FILTER(isURI(?object) || isLiteral(?object))
            FILTER(REGEX(str(?predicate), "^http://dbpedia.org/ontology/[^/]+$"))
            FILTER(!contains(str(?predicate), "wikiPageWikiLink"))
            FILTER(!contains(str(?predicate), "abstract"))
            FILTER(!contains(str(?predicate), "thumbnail"))
            FILTER(!contains(str(?predicate), "wikiPageID"))
            FILTER(!contains(str(?predicate), "wikiPageRevisionID"))
            FILTER(!contains(str(?predicate), "wikiPageLength"))
            FILTER(!contains(str(?predicate), "birthDate"))
            FILTER(!contains(str(?predicate), "birthYear"))
            FILTER(!contains(str(?predicate), "birthPlace"))
            FILTER(!contains(str(?predicate), "deathPlace"))
        }} GROUP BY ?entity
        LIMIT {limit} OFFSET {offset}
        """
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()  # Exécution de la requête et conversion des résultats en JSON

        if not results["results"]["bindings"]:
            break
        for result in results["results"]["bindings"]:
            itemset = result["itemset"]["value"].split(",")  # Récupération de l'ensemble des prédicats
            transactions.append(itemset)
        offset += limit  # Incrémentation du décalage pour la pagination
        time.sleep(1)  # Pour éviter de surcharger le serveur

    print("Data extraction complete.")
    # Création d'un DataFrame pandas à partir des transactions
    df = pd.DataFrame(transactions)
    df.to_csv('transactions.csv', index=False)
    return transactions  # Retourne la liste des transactions

print("Starting main process...")
transactions = extract_data()


Starting main process...
Extracting data from offset 0...
Extracting data from offset 10000...
Extracting data from offset 20000...
Extracting data from offset 30000...
Extracting data from offset 40000...
Extracting data from offset 50000...
Extracting data from offset 60000...
Extracting data from offset 70000...
Data extraction complete.


In [ ]:
transactions

[['http://dbpedia.org/ontology/careerStation',
  'http://dbpedia.org/ontology/height',
  'http://dbpedia.org/ontology/number',
  'http://dbpedia.org/ontology/position',
  'http://dbpedia.org/ontology/team'],
 ['http://dbpedia.org/ontology/careerStation',
  'http://dbpedia.org/ontology/height',
  'http://dbpedia.org/ontology/position',
  'http://dbpedia.org/ontology/team',
  'http://dbpedia.org/ontology/wikiPageExternalLink'],
 ['http://dbpedia.org/ontology/wikiPageExternalLink'],
 ['http://dbpedia.org/ontology/deathDate',
  'http://dbpedia.org/ontology/debutTeam',
  'http://dbpedia.org/ontology/position'],
 ['http://dbpedia.org/ontology/careerStation',
  'http://dbpedia.org/ontology/height',
  'http://dbpedia.org/ontology/number',
  'http://dbpedia.org/ontology/position',
  'http://dbpedia.org/ontology/team',
  'http://dbpedia.org/ontology/wikiPageExternalLink'],
 ['http://dbpedia.org/ontology/team',
  'http://dbpedia.org/ontology/wikiPageExternalLink'],
 ['http://dbpedia.org/ontology/

On crée ensuite un fichier de transactions au format requis par l'outil SPMF grace aux transactions précédentes en attribuant à chaque item un identifiant unique.

In [ ]:
def create_transactions_file(transactions, filename):
    print("Creating transactions file...")
    item_to_id = {}  # Dictionnaire pour mapper les items à des identifiants
    id_to_item = {}  # Dictionnaire pour mapper les identifiants aux items
    next_id = 1  # Identifiant initial
    for transaction in transactions:
        for item in transaction:
            if item not in item_to_id:
                item_to_id[item] = str(next_id)  # Assignation d'un nouvel identifiant à l'item
                id_to_item[str(next_id)] = item  # Mappage de l'identifiant à l'item correspondant
                next_id += 1
    with open(filename, 'w') as f:
        for transaction in transactions:
            transaction_ids = [item_to_id[item] for item in transaction]  # Conversion des items en identifiants
            f.write(' '.join(set(transaction_ids)) + '\n')
    print(f"Transactions file '{filename}' created.")
    return id_to_item  # Retourne le dictionnaire de mappage des identifiants aux items

id_to_item = create_transactions_file(transactions, 'transactions.txt')


Creating transactions file...
Transactions file 'transactions.txt' created.


On fournit le fichier qu'on a obtenu en input pour spmf et on génére nos rules avec l'algo 'FPGrowth_association_rules_with_lift'


In [ ]:
file_path = 'spmf_output.txt'

if not os.path.exists(file_path):
    with open(file_path, 'w') as file:
        file.write('')

def run_spmf_fpgrowth(transactions_file="transactions.txt", output_file="spmf_output.txt", min_support=0.01, min_confidence=0.8, min_lift=1.5):
    print("Running FPGrowth with SPMF...")
    command = f"java -jar {SPMF_PATH} run FPGrowth_association_rules_with_lift {transactions_file} {output_file} {min_support} {min_confidence} {min_lift}"
    print("execution de la commande")
    os.system(command)
    print("FPGrowth with SPMF completed.")

run_spmf_fpgrowth()


Running FPGrowth with SPMF...
execution de la commande
FPGrowth with SPMF completed.


On parse le fichier d'output de SPMF afin de pouvoir générer nos requêtes ensuite

In [ ]:
def parse_spmf_output(spmf_output_file, id_to_item):
    print("Parsing SPMF output...")
    rules = []  # Liste pour stocker les règles extraites
    with open(spmf_output_file, 'r') as f:
        for line in f:
            if line.startswith('#'): continue
            parts = line.strip().split('#')
            rule_parts = parts[0].split('==>')  # Séparation de la règle en parties antécédente et conséquente
            antecedent = [id_to_item[i] for i in rule_parts[0].strip().split(' ')]  # Conversion de l'antécédent en items
            consequent = [id_to_item[i] for i in rule_parts[1].strip().split(' ')]  # Conversion du conséquent en items
            support = int(parts[1].split(':')[1].strip())
            confidence = float(parts[2].split(':')[1].strip())
            lift = float(parts[3].split(':')[1].strip())
            rules.append((antecedent, consequent, support, confidence, lift))  # Ajout de la règle à la liste
    print("SPMF output parsed.")
    return rules  # Retourne la liste des règles extraites

spmf_output_file = "spmf_output.txt"

rules = parse_spmf_output(spmf_output_file, id_to_item)


Parsing SPMF output...
SPMF output parsed.


On génère les requêtes avec les règles obtenus  

In [ ]:
import csv

def generate_queries(rules, id_to_item):
    print("Generating SPARQL queries from association rules...")
    queries = []
    for antecedent, consequent, support, confidence, lift in rules:
        ant_counter = 1  # Compteur pour l'antécédent
        cons_counter = 1  # Compteur pour le conséquent

        # Construction des prédicats pour l'antécédent
        antecedent_predicates = ' . '.join([f'?subject dbo:{p.split("http://dbpedia.org/ontology/")[1]} ?ant{ant_counter + i}' for i, p in enumerate(antecedent)])
        ant_counter += len(antecedent)

        # Construction des prédicats pour le conséquent
        consequent_predicates = ' . '.join([f'?subject dbo:{p.split("http://dbpedia.org/ontology/")[1]} ?cons{cons_counter + i}' for i, p in enumerate(consequent)])
        cons_counter += len(consequent)

        # Requête SPARQL
        query = f"""
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {{
            {antecedent_predicates} .
            {consequent_predicates} .
        }}
        LIMIT 100
        """
        queries.append((query, confidence, lift, support))  # Ajout de la requête à la liste des requêtes avec ses métriques
    print("SPARQL query generation complete.")


    with open('queries.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Query", "Confidence", "Lift", "Support"])  # Écriture de l'en-tête du fichier CSV
        for query in queries:
            writer.writerow(query)

    return queries  # Retourne la liste des requêtes générées

queries = generate_queries(rules, id_to_item)


Generating SPARQL queries from association rules...
SPARQL query generation complete.


Nous allons sélectionner uniquement les 10 requêtes les plus fréquentes, basées sur leur support. Nous choisirons également d'exclure les doublons, c'est-à-dire les requêtes qui récupèrent les mêmes résultats qu'une autre requête, où seule la permutation de l'ordre des prédicats varie.

In [ ]:
def get_top_queries(queries, top_n=10):
    print(f"Getting top {top_n} queries...")
    sorted_queries = sorted(queries, key=lambda x: x[3], reverse=True)  # Tri des requêtes par support décroissant

    unique_queries = []  # Liste pour stocker les meilleures requêtes uniques
    unique_predicates = set()  # Ensemble pour stocker les prédicats uniques

    for query, confidence, lift, support in sorted_queries:
        predicates = set(extract_predicates_from_query(query))

        # Vérification si les prédicats de la requête sont tous nouveaux
        if not predicates.issubset(unique_predicates):
            unique_queries.append((query, confidence, lift, support))
            unique_predicates.update(predicates)  # Mise à jour des prédicats uniques

        if len(unique_queries) == top_n:  # Arrêt dès qu'on atteint le nombre requis de requêtes uniques
            break

    print(f"Top {top_n} unique queries obtained.")
    return unique_queries

def extract_predicates_from_query(query):
    predicates = []
    for line in query.split('\n'):
        if 'dbo:' in line:
            predicate = line.split('dbo:')[1].split(' ')[0]
            predicates.append(predicate)
    return predicates  # Retourne la liste des prédicats extraits

top_queries=get_top_queries(queries)


Getting top 10 queries...
Top 10 unique queries obtained.


On valide les requêtes en vérifiant qu'elles ne return pas d'erreurs et en vérifiant de plus qu'elle return bien au moins un résultat

In [ ]:


def validate_queries(queries):
    print("Validating generated SPARQL queries...")
    requete_non_valide = 0
    requete_valide = 0
    requete_avec_resultats = 0
    validated_queries = []
    for query, confidence, lift, support in queries:
        sparql = SPARQLWrapper(DBPEDIA_ENDPOINT)
        sparql.setQuery(query)  # Définition de la requête SPARQL
        sparql.setReturnFormat(JSON)
        try:
            response = sparql.query().convert()
            num_results = len(response["results"]["bindings"])
            if num_results > 0:  # Vérification si la requête a renvoyé des résultats
                print(f"Query '{query}' returned {num_results} results.")
                validated_queries.append((query, confidence, lift, support))
                requete_valide += 1
                requete_avec_resultats += 1
            else:
                print(f"Query '{query}' is valid but returned no results.")
                requete_valide += 1
        except Exception as e:
            print(f"Query '{query}' is not valid: {e}")
            requete_non_valide += 1
    print("SPARQL query validation complete.")
    print(f"Nombre de requêtes valides : {requete_valide}")
    print(f"Nombre de requêtes non valides : {requete_non_valide}")
    print(f"Nombre de requêtes valides avec résultats : {requete_avec_resultats}")
    return validated_queries


validated_queries = validate_queries(top_queries)


Validating generated SPARQL queries...
Query '
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:careerStation ?ant1 .
            ?subject dbo:position ?cons1 .
        }
        LIMIT 100
        ' returned 100 results.
Query '
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:team ?ant1 .
            ?subject dbo:position ?cons1 .
        }
        LIMIT 100
        ' returned 100 results.
Query '
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:height ?ant1 . ?subject dbo:position ?ant2 .
            ?subject dbo:careerStation ?cons1 .
        }
        LIMIT 100
        ' returned 100 results.
Query '
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:party ?ant1 .
            ?subject dbo:termPeriod ?cons1 .
        }
        LIMIT 100
        

Toutes les reqûetes étant valide et avec résultat, on print ces requêtes avec leurs métriques

In [ ]:
print("Top 10 queries with their confidence and lift:")
for query, confidence, lift, support in top_queries:
    print(f"Query: {query}")
    print(f"Confidence: {confidence}")
    print(f"Lift: {lift}")
    print(f"Support: {support}")
    print()


Top 10 queries with their confidence and lift:
Query: 
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:careerStation ?ant1 .
            ?subject dbo:position ?cons1 .
        }
        LIMIT 100
        
Confidence: 0.8385025442209838
Lift: 4.5069183150713625
Support: 6921

Query: 
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:team ?ant1 .
            ?subject dbo:position ?cons1 .
        }
        LIMIT 100
        
Confidence: 0.9284275547200215
Lift: 4.99026172242814
Support: 6914

Query: 
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:height ?ant1 . ?subject dbo:position ?ant2 .
            ?subject dbo:careerStation ?cons1 .
        }
        LIMIT 100
        
Confidence: 0.8796915167095116
Lift: 5.481605740116345
Support: 5133

Query: 
        PREFIX dbo: <http://dbpedia.org/ontology/>
     

On regarde maintenant si les entités générés par ces 10 requêtes sont aussi instanciés dans wikidata en faisant une requete ask sur l'endpoint de dbpedia pour savoir si on a une propriété  owl:sameAs qui relie dbpedia a wikiddata

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

def check_wikidata_instances(top_queries):
    print("Vérification si les entités des requêtes principales sont aussi instanciées dans Wikidata...")
    entities_in_wikidata = 0
    total_entities = 0
    nb_requete = 0

    for query, confidence, lift, support in top_queries:
        dbpedia_sparql = SPARQLWrapper(DBPEDIA_ENDPOINT)
        dbpedia_sparql.setQuery(query)
        dbpedia_sparql.setReturnFormat(JSON)
        response = dbpedia_sparql.query().convert()
        print("Examen de la requête : " + query)

        c = 0
        for result in response.get("results", {}).get("bindings", []):
            entity_uri = result.get("subject", {}).get("value")
            if entity_uri:
                total_entities += 1

                wikidata_query = f"""
                ASK {{
                    <{entity_uri}> owl:sameAs ?wikidata_uri .
                    FILTER(REGEX(STR(?wikidata_uri), "^http://www.wikidata.org/entity/Q"))
                }}
                """
                wikidata_sparql = SPARQLWrapper(DBPEDIA_ENDPOINT)
                wikidata_sparql.setQuery(wikidata_query)
                wikidata_sparql.setReturnFormat(JSON)
                wikidata_response = wikidata_sparql.query().convert()

                if wikidata_response.get("boolean", False):
                    entities_in_wikidata += 1
                    c += 1

                if c == 100:
                    nb_requete += 1
                    break

    print(f"Nombre total d'entités examinées: {total_entities}")
    print(f"Nombre d'entités trouvées dans Wikidata: {entities_in_wikidata}")
    print(f"Nombre de requêtes contenant des entités liées à Wikidata: {nb_requete}")

check_wikidata_instances(top_queries)

Vérification si les entités des requêtes principales sont aussi instanciées dans Wikidata...
Examen de la requête : 
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:careerStation ?ant1 .
            ?subject dbo:position ?cons1 .
        }
        LIMIT 100
        
Examen de la requête : 
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:team ?ant1 .
            ?subject dbo:position ?cons1 .
        }
        LIMIT 100
        
Examen de la requête : 
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:height ?ant1 . ?subject dbo:position ?ant2 .
            ?subject dbo:careerStation ?cons1 .
        }
        LIMIT 100
        
Examen de la requête : 
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT * WHERE {
            ?subject dbo:party ?ant1 .
            ?subject dbo:termPer

la totalité des entités sont égalements instanciés dans wikidata

On adaptes les prédicats des top 10 requêtes pour vérifier également si nos requêtes marchent correctement avec wikidata

---



In [ ]:
queries_updated = [
    # Query 1: dbo:careerStation, dbo:position
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P54 ?ant1 .   # dbo:careerStation -> wdt:P54 (team)
        ?subject wdt:P413 ?cons1 .  # dbo:position -> wdt:P413 (position played on team)
    }
    LIMIT 100
    """,
    # Query 2: dbo:team, dbo:position
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P54 ?ant1 .   # dbo:team -> wdt:P54 (team)
        ?subject wdt:P413 ?cons1 . # dbo:position -> wdt:P413 (position played on team)
    }
    LIMIT 100
    """,
    # Query 3: dbo:height, dbo:position, dbo:careerStation
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P2048 ?ant1 .  # dbo:height -> wdt:P2048 (height)
        ?subject wdt:P413 ?ant2 .   # dbo:position -> wdt:P413 (position played on team)
        ?subject wdt:P54 ?cons1 .   # dbo:careerStation -> wdt:P54 (team)
    }
    LIMIT 100
    """,
    # Query 4: dbo:party, dbo:termPeriod
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P102 ?ant1 .  # dbo:party -> wdt:P102 (political party)
        ?subject wdt:P768 ?cons1 . # dbo:termPeriod -> wdt:P768 (electoral district)
    }
    LIMIT 100
    """,
    # Query 5: dbo:deathYear, dbo:deathDate
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P570 ?ant1 .  # dbo:deathYear -> wdt:P570 (date of death)
        ?subject wdt:P20 ?cons1 .  # dbo:deathDate -> wdt:P20 (place of death)
    }
    LIMIT 100
    """,
    # Query 6: dbo:activeYearsEndYear, dbo:activeYearsStartYear
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P2031 ?ant1 .  # dbo:activeYearsEndYear -> wdt:P2031 (end time of career)
        ?subject wdt:P2032 ?cons1 . # dbo:activeYearsStartYear -> wdt:P2032 (start time of career)
    }
    LIMIT 100
    """,
    # Query 7: dbo:weight, dbo:height
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P2067 ?ant1 .  # dbo:weight -> wdt:P2067 (mass)
        ?subject wdt:P2048 ?cons1 . # dbo:height -> wdt:P2048 (height)
    }
    LIMIT 100
    """,
    # Query 8: dbo:number, dbo:position
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P161 ?ant1 .   # dbo:number -> wdt:P161 (cast member, assumed placeholder)
        ?subject wdt:P413 ?cons1 .  # dbo:position -> wdt:P413 (position played on team)
    }
    LIMIT 100
    """,
    # Query 9: dbo:wikiPageExternalLink, dbo:deathYear, dbo:deathDate
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P856 ?ant1 .   # dbo:wikiPageExternalLink -> wdt:P856 (official website)
        ?subject wdt:P570 ?ant2 .   # dbo:deathYear -> wdt:P570 (date of death)
        ?subject wdt:P20 ?cons1 .   # dbo:deathDate -> wdt:P20 (place of death)
    }
    LIMIT 100
    """,
    # Query 10: dbo:stateOfOrigin, dbo:nationality
    """
    SELECT (COUNT(DISTINCT ?subject) AS ?count) WHERE {
        ?subject wdt:P495 ?ant1 .  # dbo:stateOfOrigin -> wdt:P495 (country of origin)
        ?subject wdt:P27 ?cons1 .  # dbo:nationality -> wdt:P27 (nationality)
    }
    LIMIT 100
    """
]


on execute les requetes et on print le nombre de résultat

In [ ]:

def execute_query(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"][0]["count"]["value"]

for i, query in enumerate(queries_updated, 1):
    result_count = execute_query(WIKIDATA_ENDPOINT, query)
    print(f"Query {i}: {result_count} entities")

Query 1: 339304 entities
Query 2: 339304 entities
Query 3: 107955 entities
Query 4: 292 entities
Query 5: 1393027 entities
Query 6: 68635 entities
Query 7: 152404 entities
Query 8: 0 entities
Query 9: 13113 entities
Query 10: 928 entities


Seul une des requêtes ne renvoit pas d'entités, surement une erreur dans la transformation de la requête pour wikidata ou alors il n' y a pas d'entité correspondante dans wikidata

On créé pour finir un graphe pour mieux visualiser les règles d'associations

In [ ]:
import pandas as pd
import altair as alt

def create_rules_graph(rules, top_n=20):
    print(f"Creating graph of top {top_n} association rules...")

    sorted_rules = sorted(rules, key=lambda x: x[2], reverse=True)[:top_n]  # Tri des règles par lift décroissant et sélection des top_n premières

    df = pd.DataFrame(sorted_rules, columns=['antecedent', 'consequent', 'support', 'confidence', 'lift'])  # Création d'un DataFrame à partir des règles sélectionnées

    def clean_prefix(values):
        return [val.replace("http://dbpedia.org/ontology/", "") for val in values]

    df['antecedent'] = df['antecedent'].apply(lambda x: clean_prefix(x))  # Nettoyage des préfixes dans l'antécédent
    df['consequent'] = df['consequent'].apply(lambda x: clean_prefix(x))  # Nettoyage des préfixes dans le conséquent

    df['antecedent'] = df['antecedent'].apply(lambda x: ', '.join(x))  # Conversion de la liste d'antécédents en chaîne de caractères
    df['consequent'] = df['consequent'].apply(lambda x: ', '.join(x))  # Conversion de la liste de conséquents en chaîne de caractères

    df['rule'] = df['antecedent'] + ' -> ' + df['consequent']  # Création d'une nouvelle colonne contenant la règle complète

    graph = alt.Chart(df).mark_bar().encode(
        x='support',
        y=alt.Y('rule', sort=alt.EncodingSortField(field='support', order='descending')),
        color=alt.Color('lift', scale=alt.Scale(scheme='blues')),
        tooltip=['rule', 'support', 'confidence', 'lift']
    ).properties(
        title=f'Top {top_n} Association Rules',
        width=800,
        height=600
    )

    graph.display()

create_rules_graph(rules)


Creating graph of top 20 association rules...


alt.Chart(...)